In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'C:\Users\utente\Documents\GitHub\Project2'
sys.path.insert(0,my_path + r'/code/Flavio')

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
print('TensorFlow imported')

import dictionary_helpers as dh

# imports
import numpy as np 
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TensorFlow imported


In [7]:
import os
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_stanford')

glove_dict = dh.build_glove_dict('glove.twitter.27B.25d.txt')

print(len(glove_dict))


1193514


In [8]:
glove_dict[',']

array([ 0.84705 , -1.0349  , -0.050419,  0.27164 , -0.58659 ,  0.99514 ,
        0.25267 ,  1.6963  ,  0.10313 ,  0.80073 ,  0.74655 , -1.2667  ,
       -4.036   , -0.22557 ,  0.16322 , -0.67015 , -0.64812 ,  0.010373,
       -0.71889 , -0.74997 ,  0.24862 ,  0.10319 , -1.1732  ,  0.58196 ,
        0.33846 ])

In [9]:
import os
#os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\short')
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\full')

tweets = dh.build_tweets_matrix('all_full_processed.txt')
        
print(len(tweets))

2280482


In [10]:
tweets[1][1]

'your'

In [11]:
x = np.array([])

ciao=np.append(x,[1,4], axis=0)


print(ciao)

[ 1.  4.]


### Build averages+min+max

In [12]:
feat_size = 25

In [ ]:
X = np.empty([len(tweets), feat_size*3])

for i in range(len(tweets)):
    feats = []
    for j in range(len(tweets[i])):
        if tweets[i][j] in glove_dict:
            feats.append(glove_dict[tweets[i][j]])
        else:
            feats.append([0]*feat_size)

    X[i]= np.hstack((np.mean(feats, axis=0), np.amax(feats, axis=0), np.amin(feats, axis=0)))

print(X.shape)
print(X[9])

labels = []

for i in range(1127644):
    labels.append([1,0])
for i in range(len(tweets)-1127644):
    labels.append([0,1])


(2280482, 75)
[-0.1865898  0.236394   0.330381  -0.508172  -0.6313082 -0.1213108
  1.256084  -0.3280116 -0.69894    0.190782  -0.289562   0.6072632 -5.21422
 -0.1280236 -0.211636   0.387608   0.2012168 -0.0554106 -0.053988
 -0.2539778  0.139662   0.33784   -0.0941022 -0.0355024  0.0722898
  0.088601   0.59108    0.96279    0.19839   -0.092441   0.38567    2.1
  0.066642  -0.49794    0.70047    0.43174    1.4784    -3.1591     0.15437
  0.52152    0.86374    0.59554    0.88961    1.292      0.33094    0.4516
  0.65844    0.54594    0.25164    0.2758    -0.45649   -0.20207   -0.12672
 -1.1372    -1.247     -0.40767    0.68192   -0.84406   -1.1467    -0.17034
 -0.87221   -0.013954  -5.995     -0.43106   -0.9154    -0.11314   -0.33704
 -0.47587   -0.84178   -0.84126   -0.4582    -0.16727   -0.47606   -0.39695
 -0.16073  ]


In [ ]:
featSet = []

for i in range(len(tweets)):
    featSet.append([X[i,:].tolist(), labels[i]])

In [ ]:
print(featSet[190000])

[[0.181188, 0.13865947419354843, 0.006939490322580643, -0.2190483870967742, -0.285460806451613, -0.03187148387096773, 1.0052274193548385, 0.250002870967742, -0.2340423870967742, 0.15821677419354843, -0.28707551612903226, 0.03712477419354836, -4.454083870967742, -0.3381245161290322, 0.06274933225806452, -0.18583977419354836, 0.21532859999999998, -0.44396445161290327, -0.3509488709677419, -0.382074064516129, 0.24983261935483878, 0.2606583967741935, -0.13827661290322582, 0.36713777741935477, -0.06539945161290323, 1.6477, 1.2678, 1.0937, 1.3254, 0.74877, 0.68537, 2.1431, 2.9738, 2.1517, 1.035, 0.53267, 1.3131, 0.0, 0.91787, 1.2789, 0.78043, 1.5676, 1.8308, 1.1712, 0.94705, 1.5811, 1.3184, 1.184, 1.5532, 1.3129, -1.0206, -0.87651, -0.71516, -1.6632, -1.0503, -0.70469, -1.6111, -1.1824, -1.3219, -1.0633, -1.6912, -2.9371, -6.2619, -2.0434, -1.6743, -0.94879, -1.0932, -1.1591, -2.0487, -1.6247, -1.0907, -0.57663, -1.1142, -1.2152, -0.77178], [1, 0]]


In [ ]:
del(X)
del(labels)
del(tweets)
del(glove_dict)

#### crea train_x train_y test_x test_y

In [ ]:
import random

random.shuffle(featSet)

featSet = np.array(featSet)

testing_size = int(0.1*len(featSet))

train_x = list(featSet[:,0][:-testing_size])
train_y = list(featSet[:,1][:-testing_size])

test_x = list(featSet[:,0][-testing_size:-1])
test_y = list(featSet[:,1][-testing_size:-1])


In [ ]:
n_inputs = 75
n_nodes_hl1 = 75
n_nodes_hl2 = 30
n_nodes_hl3 = 15

n_classes = 2
batch_size = 250


# input feature size = 50feats x 20words = 2000 
x = tf.placeholder('float', [None, n_inputs])
y = tf.placeholder('float')

#DEFINITION OF THE NETWORK:
#1) random initialization of the various layer with respectives dimensions, the layer is a DICTIONARY of weigths and biases
#2) application of weights and biases and activation function to each layer


def neural_network_model(data):
    # input_data * weights + biases

    #tf.random_normal returns an array of the specified dimension filled with randomly distributed numbers
    #tf.Variable = object to be created that works in the tensorflow session, just a wrapper.
    hidden_l1 = {'weights': tf.Variable(tf.random_normal([n_inputs, n_nodes_hl1])),
    'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_l2 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
    'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_l3 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
    'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_l  = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
    'biases': tf.Variable(tf.random_normal([n_classes]))}

    l1 = tf.add(tf.matmul(data, hidden_l1['weights']), hidden_l1['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1, hidden_l2['weights']), hidden_l2['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_l3['weights']), hidden_l3['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.add(tf.matmul(l3, output_l['weights']), output_l['biases'])
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))   # v1.0 changes
    # optimizer value = 0.001, Adam similar to SGD
    # everything is updating through this optimizer, i don't know how.... problem!? maybe better not to know
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    epochs_no = 50
 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())   # v1.0 changes
  
        # training
        for epoch in range(epochs_no):
            epoch_loss = 0

            #batching
            i = 0
            while i < len(train_x):
                start = i
                end = i+batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                # code that optimizes the weights & biases
                epoch_loss += c
                i+=batch_size
                
            print('Epoch', epoch+1, 'completed out of', epochs_no, 'loss:', epoch_loss)

        # testing
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_x, y: test_y}))

train_neural_network(x)

## TESTTTTT

In [ ]:
def test_neural_network():
    prediction = neural_network_model(x)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
      
        for epoch in range(hm_epochs):
            try:
                saver.restore(sess,"model.ckpt")
            except Exception as e:
                print(str(e))
            epoch_loss = 0


        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        feature_sets = []
        labels = []
        counter = 0
        with open('processed-test-set.csv', buffering=20000) as f:
            for line in f:
                try:
                    features = list(eval(line.split('::')[0]))
                    label = list(eval(line.split('::')[1]))

                    #print(features)
                    #print(label)

                    feature_sets.append(features)
                    labels.append(label)
                    counter += 1
                except:
                    pass
        print('Tested',counter,'samples.')

        test_x = np.array(feature_sets)
        test_y = np.array(labels)
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))


test_neural_network()